In [3]:
using Pkg
Pkg.activate("psyc261")
Pkg.add("Conda")
using Conda
Conda.pip_interop(true)  # This allows Conda to use `pip`
Conda.pip("install", "mitsuba")
Conda.add("numpy")

Pkg.add(["Distributions", "ProgressMeter", "Parameters", "JLD2"])
using JLD2
using Distributions
using ProgressMeter
using Parameters
using Random
using Gen, Plots
using PyCall
np = pyimport("numpy")
# to resize images (without any external packages)
include("utils/imageresize.jl");
# truncated normal 
include("utils/truncatednorm.jl")

  Activating project at `~/psyc261`
   Resolving package versions...
  No Changes to `~/psyc261/Project.toml`
  No Changes to `~/psyc261/Manifest.toml`
[ Info: Downloading miniconda installer ...


PREFIX=/opt/julia/conda/3/x86_64


[ Info: Installing miniconda ...
/tmp/installer.sh: 302: cannot create /opt/julia/conda/3/x86_64/conda.exe: Permission denied


LoadError: failed process: Process(`[4m/tmp/installer.sh[24m [4m-b[24m [4m-f[24m [4m-p[24m [4m/opt/julia/conda/3/x86_64[24m`, ProcessExited(2)) [2]


## Mitsuba Setup ##

## Generating Training Images ##

## Inference of Latent Parameters for Images -- One-Shot Latents of Shape 3D Memories ##

In [2]:
#load dataset
py"dataset = h5py.File('3dshapes.h5', 'r')
print(dataset.keys())
images = dataset['images']  # array shape [480000,64,64,3], uint8 in range(256)
labels = dataset['labels']  # array shape [480000,6], float64
image_shape = images.shape[1:]  # [64,64,3]
label_shape = labels.shape[1:]  # [6]
n_samples = labels.shape[0]  # 10*10*10*8*4*15=480000

_FACTORS_IN_ORDER = ['floor_hue', 'wall_hue', 'object_hue', 'scale', 'shape',
                     'orientation']
_NUM_VALUES_PER_FACTOR = {'floor_hue': 10, 'wall_hue': 10, 'object_hue': 10, 
                          'scale': 8, 'shape': 4, 'orientation': 15}"


py"def show_images_grid(imgs_, num_images=25):
  ncols = int(np.ceil(num_images**0.5))
  nrows = int(np.ceil(num_images / ncols))
  _, axes = plt.subplots(ncols, nrows, figsize=(nrows * 3, ncols * 3))
  axes = axes.flatten()

  for ax_i, ax in enumerate(axes):
    if ax_i < num_images:
      ax.imshow(imgs_[ax_i], cmap='Greys_r', interpolation='nearest')
      ax.set_xticks([])
      ax.set_yticks([])
    else:
      ax.axis('off')"

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[2]:1

In [2]:
@gen function shape3d()
    #Priors
    floor hue ~ uniform(0, 1)
    wall hue ~ uniform(0, 1)
    object hue ~ uniform(0, 1)
    scale ~ uniform(0, 1)
    shape ~ draw([0, 1, 2, 3])

    #Likelihood
    pred ~ broadcasted_normal(mu, 1.0)

end
;

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[2]:1

In [2]:
#not sure what proposal distribution should be 
function proposal(tr)
   ##
end

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[2]:1

In [2]:
#basic mh algorithm
function random_walk_mh(tr)
    # make a random-walk update on scale variables
    (tr, accepted) = mh(tr, proposal, ())
    return tr
end
;

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[2]:1

In [2]:
function do_inference()
    K = 10000
    t, = generate(room, (), obs_image)
    scores = Vector{Float64}(undef, K)
    @showprogress for i in 1:K
        t = random_walk_mh(t)
        scores[i] = get_score(t)
    end;
    return scores, t
end
;

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[2]:1

In [2]:
(scores, trace) = do_inference()

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[2]:1

## Generating Stimulus Images ##

## Unfolding (Inference of Latent Parameters for Stimuli and Recall of Memory) ##

In [2]:
struct Memory_State
    current_memory::Int64
    latent_memories::Matrix{Float64}
    consolidation::Vector{Float64}
end

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[2]:1

In [1]:
function reset_consolidation_vector!(m_state::Memory_State)
    fill!(m_state.consolidation_vector, 1.0)
end

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[1]:1

In [1]:
function add_noise!(matrix, row_index, noise_level)
    row = view(matrix, row_index, :)

    for i in eachindex(row)
        noise = (rand() * 0.2 - 0.1) * noise_level * row[i] 
        row[i] += noise
    end
end


function draw_consolidation_vector!(m_state::Memory_State)
    fill!(m_state.consolidation_vector, 1.0)
end

LoadError: LoadError: UndefVarError: `@with_kw` not defined
in expression starting at In[1]:1

In [4]:
function consolidate!(m_state::Memory_State, memory_recalled::Int64, recall_noise::Float64)
    m_state.current_memory = memory_recalled
    m_state.consolidation_vector[m_state.current_memory] = m_state.consolidation_vector[m_state.current_memory] + 1
    add_noise!(m_state.latent_memories, memory_recalled, recall_noise)
end

  Activating project at `~/psyc261`
   Resolving package versions...
  No Changes to `~/psyc261/Project.toml`
  No Changes to `~/psyc261/Manifest.toml`


LoadError: PyError (PyImport_ImportModule

The Python package numpy could not be imported by pyimport. Usually this means
that you did not install numpy in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/opt/conda/bin/python3

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the numpy module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg.build("PyCall"), and re-la

In [ ]:
function get_similar_memory(matrix, stimulus_vector, cons_factor)
    distances = [(sum(abs.(row .- stimulus_vector))) for row in eachrow(matrix)]
    remembered = distances/cons_factor
    memory =  draw(softmax(remembered))
    return memory
end

 function recall_memory(memory_state::Memory_State, stimulus_vector::Float64)
        prob_vector = softmax(memory_state.consolidation)
        recalled_memory = get_similar_memory(memory_state.latent_memories, stimulus_vector, prob_vector)
        return recalled_memory
 end


In [ ]:
@gen (static) function kernel(t::Int, prev_memory_state::Memory_State, recall_noise::Float64)
    stim_vector = do_inference(image::scene)
    memory_recalled = recall_memory(prev_memory_state, stim_vector)
    next_state = consolidate!(prev_memory_state, memory_recalled, recall_noise)
    return next_state
end

chain = Gen.Unfold(kernel)


In [ ]:
(trace, _) = Gen.generate(unfold_model, (200,))
Gen.get_choices(trace)